In [16]:
import csv
import numpy as np
import math 
import os
import numpy as np
import datetime
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation
from tensorflow.keras.utils import Sequence

In [3]:
def angle(A, B, C):
    AB = (B[0] - A[0], B[1] - A[1])
    BC = (C[0] - B[0], C[1] - B[1])

    dot_product = AB[0] * BC[0] + AB[1] * BC[1]
    magnitude_AB = math.sqrt(AB[0] ** 2 + AB[1] ** 2)
    magnitude_BC = math.sqrt(BC[0] ** 2 + BC[1] ** 2)

    cos_theta = dot_product / (magnitude_AB * magnitude_BC)

    # Calculate the acute angle in radians
    theta = math.acos(cos_theta)

    # Convert the angle to degrees
    theta_degrees = math.degrees(theta)

    return theta_degrees

In [4]:
list_of_csv_right = ["./training/correct_3.csv", 
               "./training/correct_4.csv",  
               "./training/trial_1_left_curl_correct.csv",
               "./training/trial_2_left_curl_correct.csv",
            ]

list_of_csv_wrong = [
               "./training/angle_small_1.csv",
               "./training/angle_small_2.csv",
               "./training/wrong_angle_too_small.csv",
               "./training/down_too_fast_1.csv",
               "./training/down_too_fast_2.csv",
               "./training/trial_3_left_curl_wrong_too_fast_down.csv",
               "./training/too_light_1.csv",
               "./training/too_light_2.csv",
               "./training/trial_4_left_curl_wrong_too_light.csv",
]

In [5]:
def parse_csv(path):
    arr = []
    # Specify the path to your CSV file
    csv_file_path = path

    # Open and read the CSV file
    with open(csv_file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        
        # Iterate through each row in the CSV file
        counter = 0
        repcount = 0
        repcountfour = 1
        for row in reader:
            tempcount = counter
            if counter >=17:
                reps = 17*repcountfour
                tempcount = counter - reps
            if tempcount == 5 or tempcount == 7 or tempcount == 9 or tempcount ==11:
                repcount+=1
                row[1] = float(row[1])
                row[2] = float(row[2])
                #print(row)
                arr.append(row)
            if repcount == 4:
                repcountfour +=1
                repcount = 0
            counter+=1
    angle_change_right = np.array(arr,dtype=object)
    #print(arr)
    return arr
                

In [6]:
def consec_decreasing(arr, index):
    if len(arr) - index >= 5:
        for i in range(index + 1, index + 5):
            if arr[i] >= arr[i - 1]:
                return False
        return True
    else:
        return False

def consec_increasing(arr, index):
    # Check if there are at least 4 elements including the specified index
    if len(arr) - index >= 5:
        for i in range(index + 1, index + 5):
            if arr[i] <= arr[i - 1]:
                return False
        return True
    else:
        return False
    
def adj_duplicates(arr):
    duplicates = []
    for i in range(len(arr) - 1):
        if arr[i][2] == arr[i + 1][2]:
            duplicates.append(i)

    return duplicates

In [7]:
print(len(list_of_csv_wrong))

9


In [8]:
dataset_right = []

for i in list_of_csv_right:
    temparr = parse_csv(i)
    dataset_right.append(temparr)
    

dataset_wrong = []
dataset_wrong2 = []
dataset_wrong3 = []

for i in range(3):
    path = list_of_csv_wrong[i]
    temparr = parse_csv(path)
    dataset_wrong.append(temparr)

for i in range(3,6):
    path = list_of_csv_wrong[i]
    temparr = parse_csv(path)
    dataset_wrong2.append(temparr)

for i in range(6,9):
    path = list_of_csv_wrong[i]
    temparr = parse_csv(path)
    dataset_wrong3.append(temparr)
    
dataset_right = np.concatenate(dataset_right)
dataset_wrong = np.concatenate(dataset_wrong)
dataset_wrong2 = np.concatenate(dataset_wrong2)
dataset_wrong3 = np.concatenate(dataset_wrong3)


# Calculate the number of groups (x)
x = dataset_wrong.shape[0] // 4
# Reshape the data to (x, 4, 3)
dataset_wrong = dataset_wrong[:x*4].reshape(x, 4, 3)
x = dataset_wrong2.shape[0] // 4
dataset_wrong2 = dataset_wrong2[:x*4].reshape(x, 4, 3)
x = dataset_wrong3.shape[0] // 4
dataset_wrong3 = dataset_wrong3[:x*4].reshape(x, 4, 3)


# Calculate the number of groups (x)
x = dataset_right.shape[0] // 4
# Reshape the data to (x, 4, 3)
dataset_right = dataset_right[:x*4].reshape(x, 4, 3)
dataset_right = np.array(dataset_right,dtype=object)
dataset_wrong = np.array(dataset_wrong,dtype=object)
dataset_wrong2 = np.array(dataset_wrong2,dtype=object)
dataset_wrong3 = np.array(dataset_wrong3,dtype=object)
dataset_right[:, :, 1:] = dataset_right[:, :, 1:].astype(float)
dataset_wrong[:, :, 1:] = dataset_wrong[:, :, 1:].astype(float)
dataset_wrong2[:, :, 1:] = dataset_wrong2[:, :, 1:].astype(float)
dataset_wrong3[:, :, 1:] = dataset_wrong3[:, :, 1:].astype(float)

In [9]:
curl_angles_right = []
for frame in dataset_right:
    A = (frame[0][1],frame[0][2])
    B = (frame[1][1],frame[1][2])
    C = (frame[2][1],frame[2][2])
    curl_angles_right.append(angle(A,B,C))
    
angle_noise_threshold = 0
#angle_change_right = change(curl_angles_right)
changes = []
increasing = False
if curl_angles_right[1] - curl_angles_right[0] > 0:
        increasing = True
for i in range(1, len(curl_angles_right)):
    diff = curl_angles_right[i] - curl_angles_right[i - 1]
    if diff < 0 and increasing == True and consec_decreasing(curl_angles_right,i-1): #and abs(diff) > threshold:
        changes.append(([curl_angles_right[i-1],curl_angles_right[i],curl_angles_right[i+1]],i-1,"max"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"max"))
        increasing = False
    elif diff > 0 and increasing == False and consec_increasing(curl_angles_right,i-1): #and abs(diff) > threshold:
        changes.append(([curl_angles_right[i-1],curl_angles_right[i],curl_angles_right[i+1]],i-1,"min"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"min"))
        increasing = True
for i in range(len(changes) - 1, -1, -1):
    if changes[i][2] == "min" and changes[i][0][0] > 130:
        changes.remove(changes[i])
    if changes[i][2] == "max" and changes[i][0][0] < 50:
        changes.remove(changes[i])
print(adj_duplicates(changes))
print(changes)
time_up = []
time_down = []
total_time = []
max_min_pairs = []
right_parameters = []
for i in range(2,len(changes),2):
    if changes[i][2] == "max" and changes[i-1][2] == "min" and changes[i-2][2] == "max":
        #total_time.append(changes[i][1]-changes[i-2][1])
        #max_min_pairs.append()
        right_parameters.append([changes[i][1]-changes[i-2][1],changes[i-2][0][0],changes[i-1][0][0]])
    elif changes[i][2] == "min" and changes[i-1][2] == "max" and changes[i-2][2] == "min":
        right_parameters.append([changes[i][1]-changes[i-2][1],changes[i-1][0][0],changes[i-2][0][0]])
print("right parameters:")
print(right_parameters)
print(len(right_parameters))

[]
[([21.780127482534386, 24.511727066095887, 25.683732315335792], 16, 'min'), ([148.0498891527402, 144.53691163273174, 139.39674011937345], 40, 'max'), ([52.370267723660724, 62.1768598169195, 81.62237973857101], 67, 'min'), ([147.013515566751, 142.85411042945137, 138.81453947226433], 82, 'max'), ([4.294440285804189, 5.717300680665785, 8.342777760315949], 99, 'min'), ([146.32775877160444, 142.63688422868225, 134.50796747030458], 123, 'max'), ([18.170072467983914, 21.247680113363607, 22.438167965992154], 150, 'min'), ([153.86124504402042, 151.90154480442146, 142.32039112684737], 176, 'max'), ([28.08084039728898, 35.81502076163649, 44.9394677561766], 207, 'min'), ([121.54144281826622, 114.1606291606582, 110.67949303398859], 236, 'max'), ([18.44989142675729, 22.439208044087145, 24.38256816441231], 259, 'min'), ([136.11781604435745, 129.33787679445138, 118.31083485918668], 286, 'max'), ([3.560707942342299, 4.86169007824623, 5.884405988022917], 307, 'min'), ([151.77121079972608, 146.9920145

In [10]:
small_angle = []
for frame in dataset_wrong:
    A = (frame[0][1],frame[0][2])
    B = (frame[1][1],frame[1][2])
    C = (frame[2][1],frame[2][2])
    small_angle.append(angle(A,B,C))

changes = []
increasing = False
if small_angle[1] - small_angle[0] > 0:
        increasing = True
for i in range(1, len(small_angle)):
    diff = small_angle[i] - small_angle[i - 1]
    if diff < 0 and increasing == True and consec_decreasing(small_angle,i-1): #and abs(diff) > threshold:
        changes.append(([small_angle[i-1],small_angle[i],small_angle[i+1]],i-1,"max"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"max"))
        increasing = False
    elif diff > 0 and increasing == False and consec_increasing(small_angle,i-1): #and abs(diff) > threshold:
        changes.append(([small_angle[i-1],small_angle[i],small_angle[i+1]],i-1,"min"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"min"))
        increasing = True
for i in range(len(changes) - 1, -1, -1):
    if changes[i][2] == "min" and changes[i][0][0] > 130:
        changes.remove(changes[i])
    if changes[i][2] == "max" and changes[i][0][0] < 50:
        changes.remove(changes[i])
print(adj_duplicates(changes))
print(changes)
time_up = []
time_down = []
total_time = []
max_min_pairs = []
wrong_parameters_1 = []
for i in range(2,len(changes),2):
    if changes[i][2] == "max" and changes[i-1][2] == "min" and changes[i-2][2] == "max":
        #total_time.append(changes[i][1]-changes[i-2][1])
        #max_min_pairs.append()
        wrong_parameters_1.append([changes[i][1]-changes[i-2][1],changes[i-2][0][0],changes[i-1][0][0]])
    elif changes[i][2] == "min" and changes[i-1][2] == "max" and changes[i-2][2] == "min":
        wrong_parameters_1.append([changes[i][1]-changes[i-2][1],changes[i-1][0][0],changes[i-2][0][0]])
print("wrong1 parameters:")
print(wrong_parameters_1)
print(len(wrong_parameters_1))

too_fast = []
for frame in dataset_wrong2:
    A = (frame[0][1],frame[0][2])
    B = (frame[1][1],frame[1][2])
    C = (frame[2][1],frame[2][2])
    too_fast.append(angle(A,B,C))


changes = []
increasing = False
if too_fast[1] - too_fast[0] > 0:
        increasing = True
for i in range(1, len(too_fast)):
    diff = too_fast[i] - too_fast[i - 1]
    if diff < 0 and increasing == True and consec_decreasing(too_fast,i-1): #and abs(diff) > threshold:
        changes.append(([too_fast[i-1],too_fast[i],too_fast[i+1]],i-1,"max"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"max"))
        increasing = False
    elif diff > 0 and increasing == False and consec_increasing(too_fast,i-1): #and abs(diff) > threshold:
        changes.append(([too_fast[i-1],too_fast[i],too_fast[i+1]],i-1,"min"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"min"))
        increasing = True
for i in range(len(changes) - 1, -1, -1):
    if changes[i][2] == "min" and changes[i][0][0] > 130:
        changes.remove(changes[i])
    if changes[i][2] == "max" and changes[i][0][0] < 50:
        changes.remove(changes[i])
print(adj_duplicates(changes))
print(changes)
time_up = []
time_down = []
total_time = []
max_min_pairs = []
wrong_parameters_2 = []
for i in range(2,len(changes),2):
    if changes[i][2] == "max" and changes[i-1][2] == "min" and changes[i-2][2] == "max":
        #total_time.append(changes[i][1]-changes[i-2][1])
        #max_min_pairs.append()
        wrong_parameters_2.append([changes[i][1]-changes[i-2][1],changes[i-2][0][0],changes[i-1][0][0]])
    elif changes[i][2] == "min" and changes[i-1][2] == "max" and changes[i-2][2] == "min":
        wrong_parameters_2.append([changes[i][1]-changes[i-2][1],changes[i-1][0][0],changes[i-2][0][0]])
print("wrong2 parameters:")
print(wrong_parameters_2)
print(len(wrong_parameters_2))

too_light = []
for frame in dataset_wrong3:
    A = (frame[0][1],frame[0][2])
    B = (frame[1][1],frame[1][2])
    C = (frame[2][1],frame[2][2])
    too_light.append(angle(A,B,C))

changes = []
increasing = False
if too_light[1] - too_light[0] > 0:
        increasing = True
for i in range(1, len(too_light)):
    diff = too_light[i] - too_light[i - 1]
    if diff < 0 and increasing == True and consec_decreasing(too_light,i-1): #and abs(diff) > threshold:
        changes.append(([too_light[i-1],too_light[i],too_light[i+1]],i-1,"max"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"max"))
        increasing = False
    elif diff > 0 and increasing == False and consec_increasing(too_light,i-1): #and abs(diff) > threshold:
        changes.append(([too_light[i-1],too_light[i],too_light[i+1]],i-1,"min"))
        #print(([arr[i-2],arr[i-1],arr[i]],i,"min"))
        increasing = True
for i in range(len(changes) - 1, -1, -1):
    if changes[i][2] == "min" and changes[i][0][0] > 130:
        changes.remove(changes[i])
    if changes[i][2] == "max" and changes[i][0][0] < 50:
        changes.remove(changes[i])
print(adj_duplicates(changes))
print(changes)
time_up = []
time_down = []
total_time = []
max_min_pairs = []
wrong_parameters_3 = []
for i in range(2,len(changes),2):
    if changes[i][2] == "max" and changes[i-1][2] == "min" and changes[i-2][2] == "max":
        #total_time.append(changes[i][1]-changes[i-2][1])
        #max_min_pairs.append()
        wrong_parameters_3.append([changes[i][1]-changes[i-2][1],changes[i-2][0][0],changes[i-1][0][0]])
    elif changes[i][2] == "min" and changes[i-1][2] == "max" and changes[i-2][2] == "min":
        wrong_parameters_3.append([changes[i][1]-changes[i-2][1],changes[i-1][0][0],changes[i-2][0][0]])
print("wrong3 parameters:")
print(wrong_parameters_3)
print(len(wrong_parameters_3))

[]
[([131.26915207977302, 127.45911697068931, 121.86148080018046], 16, 'max'), ([45.72994212153846, 50.34668045350623, 57.24246825753728], 29, 'min'), ([135.90047274123222, 128.9096488025146, 124.01385516740332], 44, 'max'), ([37.6433065044222, 45.12293816459396, 47.09260694721135], 61, 'min'), ([136.7098216682653, 133.00799976954394, 124.68180785493801], 81, 'max'), ([39.15660463598431, 40.610423805290424, 42.682404639338415], 102, 'min'), ([119.99514500215977, 118.96012559178669, 118.94111447077724], 118, 'max'), ([72.11125244667907, 82.85977421174054, 88.40114778369217], 143, 'min'), ([82.02557180028893, 74.33477104065916, 69.5638439777069], 167, 'max'), ([46.58862905277576, 51.11142968265917, 62.39731920865924], 181, 'min'), ([133.06073421250628, 126.58493212830558, 123.77242179302293], 199, 'max'), ([37.18984674544164, 39.853499939652544, 47.60630881393793], 219, 'min'), ([134.48247376866004, 130.70105235906823, 129.20798840285045], 237, 'max'), ([30.348424457095202, 32.5754750545

In [116]:
def csv_to_dataset(path):
    dataset = []
    temparr = parse_csv(path)
    dataset.append(temparr)
    dataset = np.concatenate(dataset)
    x = dataset.shape[0] // 4
    dataset = dataset[:x*4].reshape(x, 4, 3)
    dataset = np.array(dataset,dtype=object)
    dataset[:, :, 1:] = dataset[:, :, 1:].astype(float)
    curl_angles = []
    for frame in dataset:
        A = (frame[0][1],frame[0][2])
        B = (frame[1][1],frame[1][2])
        C = (frame[2][1],frame[2][2])
        curl_angles.append(angle(A,B,C))
        angle_noise_threshold = 0
    changes = []
    increasing = False
    if curl_angles[1] - curl_angles[0] > 0:
            increasing = True
    for i in range(1, len(curl_angles)):
        diff = curl_angles[i] - curl_angles[i - 1]
        if diff < 0 and increasing == True and consec_decreasing(curl_angles,i-1): #and abs(diff) > threshold:
            changes.append(([curl_angles[i-1],curl_angles[i],curl_angles[i+1]],i-1,"max"))
            #print(([arr[i-2],arr[i-1],arr[i]],i,"max"))
            increasing = False
        elif diff > 0 and increasing == False and consec_increasing(curl_angles,i-1): #and abs(diff) > threshold:
            changes.append(([curl_angles[i-1],curl_angles[i],curl_angles[i+1]],i-1,"min"))
            #print(([arr[i-2],arr[i-1],arr[i]],i,"min"))
            increasing = True
    for i in range(len(changes) - 1, -1, -1):
        if changes[i][2] == "min" and changes[i][0][0] > 130:
            changes.remove(changes[i])
        if changes[i][2] == "max" and changes[i][0][0] < 50:
            changes.remove(changes[i])
    print(adj_duplicates(changes))
    print(changes)
    parameters = []
    for i in range(2,len(changes),2):
        if changes[i][2] == "max" and changes[i-1][2] == "min" and changes[i-2][2] == "max":
            parameters.append([changes[i][1]-changes[i-2][1],changes[i-2][0][0],changes[i-1][0][0]])
        elif changes[i][2] == "min" and changes[i-1][2] == "max" and changes[i-2][2] == "min":
            parameters.append([changes[i][1]-changes[i-2][1],changes[i-1][0][0],changes[i-2][0][0]])
    #print("parameters:")
    #print(parameters)
    #print(len(parameters))
    return parameters
    

In [76]:
right_parameters = np.array(right_parameters)
wrong_small_angle = np.array(wrong_parameters_1)
wrong_down_fast = np.array(wrong_parameters_2)
wrong_too_light = np.array(wrong_parameters_3)

print(wrong_small_angle)
print(right_parameters.shape)
print(wrong_down_fast.shape)
print(wrong_too_light.shape)


[[ 28.         131.26915208  45.72994212]
 [ 37.         135.90047274  37.6433065 ]
 [ 37.         136.70982167  39.15660464]
 [ 49.         119.995145    72.11125245]
 [ 32.          82.0255718   46.58862905]
 [ 38.         133.06073421  37.18984675]
 [ 42.         134.48247377  30.34842446]
 [ 38.         123.91254033  28.90779699]
 [ 39.         117.8952003   52.69054717]
 [ 43.         121.60163553  39.02386302]
 [ 77.         119.13505096  52.00649108]
 [ 37.         127.76406268  42.16739257]
 [ 36.         121.31614225  55.16658932]
 [ 32.         123.99144433  47.16292876]
 [ 42.         134.7910544   50.92072384]
 [ 41.         133.01893938  46.62840977]
 [ 41.         122.37439317  36.04697149]
 [ 39.         131.21142517  37.13331426]
 [ 44.         124.85866228  38.59674677]
 [ 32.         122.79291067  55.23827196]
 [ 38.         127.56517868  48.54759329]
 [ 47.         140.23938278  50.06891649]
 [ 58.         122.01307614  52.87067553]
 [ 39.         122.74459141  48.45

In [31]:
# asssum datasets in size of [x,3] and 4 tiems for 4 classes
right_data = right_parameters
drop_fast_data = wrong_down_fast
too_light_data = wrong_too_light
only_small_data = wrong_small_angle


right_data_out =     [1,0,0,0]
drop_fast_data_out = [0,1,0,0]
too_light_data_out = [0,0,1,0]
only_small_data_out = [0,0,0,1]

inputs = np.concatenate((right_data,drop_fast_data,too_light_data,only_small_data,right_data,drop_fast_data,too_light_data,only_small_data,right_data)) #duplicated inputs
outputs = np.concatenate((
                        np.tile(right_data_out, (len(right_data), 1)),
                         np.tile(drop_fast_data_out, (len(drop_fast_data), 1)),
                         np.tile(too_light_data_out, (len(too_light_data), 1)),
                         np.tile(only_small_data_out, (len(only_small_data), 1)),
                         np.tile(right_data_out, (len(right_data), 1)),
                         np.tile(drop_fast_data_out, (len(drop_fast_data), 1)),
                         np.tile(too_light_data_out, (len(too_light_data), 1)),
                         np.tile(only_small_data_out, (len(only_small_data), 1)),
                         np.tile(right_data_out, (len(right_data), 1))
                         )) #duplicated outputs

#shuffle the data to prepare for training 
shuffled_indices = np.random.permutation(len(inputs))
inputs, outputs = inputs[shuffled_indices],outputs[shuffled_indices]


#create a tensorflow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((inputs,outputs))
    dataset = dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

print(inputs.shape)
print(outputs.shape)

#determine train and test 
train_data = dataset.take(int(0.7*len(dataset)))
val_data = dataset.skip(int(0.7*len(dataset)))





(649, 3)
(649, 4)


In [70]:
#define the model
input_shape = (None,3)

model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),  # Additional hidden layer
    layers.Dense(32, activation='relu'),   # Additional hidden layer
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Print the model summary to verify the input shape
model.build(input_shape)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, None, 128)         512       
                                                                 
 dense_66 (Dense)            (None, None, 64)          8256      
                                                                 
 dense_67 (Dense)            (None, None, 64)          4160      
                                                                 
 dense_68 (Dense)            (None, None, 32)          2080      
                                                                 
 dense_69 (Dense)            (None, None, 4)           132       
                                                                 
Total params: 15140 (59.14 KB)
Trainable params: 15140 (59.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
checkpoint_filepath = 'tmp/checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


#model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[model_checkpoint_callback])
model.fit(train_data, epochs=100, callbacks=[model_checkpoint_callback])
#model.fit(train_data, epochs=5)



Epoch 1/100
 1/14 [=>............................] - ETA: 2s - loss: 7.9013 - accuracy: 0.2188WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 930us/step - loss: 2.5444 - accuracy: 0.3839
Epoch 2/100
 1/14 [=>............................] - ETA: 0s - loss: 1.0748 - accuracy: 0.5312WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 802us/step - loss: 1.0738 - accuracy: 0.5513
Epoch 3/100
 1/14 [=>............................] - ETA: 0s - loss: 0.9926 - accuracy: 0.5938WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 937us/step - loss: 0.9127 - accuracy: 0.6473
Epoch 4/100
 1/14 [=>............................] - ETA: 0s - loss: 0.9166 - accuracy: 0.5938WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 990us/step - loss: 0.8434 - accuracy: 0.6830
Epoch 5/100
 1/14 [=>............................] - ETA: 0s - loss: 0.7677 - accuracy: 0.6250WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


...

14/14 [==============================] - 0s 2ms/step - loss: 0.4887 - accuracy: 0.8125
Epoch 97/100
 1/14 [=>............................] - ETA: 0s - loss: 0.5881 - accuracy: 0.7812WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 1ms/step - loss: 0.5138 - accuracy: 0.8058
Epoch 98/100
 1/14 [=>............................] - ETA: 0s - loss: 0.5265 - accuracy: 0.7812WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 1ms/step - loss: 0.4777 - accuracy: 0.8080
Epoch 99/100
 1/14 [=>............................] - ETA: 0s - loss: 0.5374 - accuracy: 0.7500WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 1ms/step - loss: 0.4763 - accuracy: 0.8214
Epoch 100/100
 1/14 [=>............................] - ETA: 0s - loss: 0.5662 - accuracy: 0.7500WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


14/14 [==============================] - 0s 1ms/step - loss: 0.4793 - accuracy: 0.8103


In [74]:
loss, accuracy = model.evaluate(val_data)
# model.save("my_model")  # This will save the model in a directory named 'my_model'
model.save("my_model")  # This will save the model in a directory named 'my_model'


7/7 [==============================] - 0s 813us/step - loss: 0.5484 - accuracy: 0.8159
INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [73]:
saved_model_path = "my_model"  # Change to the path where your model is saved
loaded_model = tf.keras.models.load_model(saved_model_path)

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("my_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/y_/6yh0ybp146v2ggxr5pqmm5mm0000gn/T/tmpvg42yt5q/assets


INFO:tensorflow:Assets written to: /var/folders/y_/6yh0ybp146v2ggxr5pqmm5mm0000gn/T/tmpvg42yt5q/assets
2023-09-17 09:16:50.573742: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-17 09:16:50.573760: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-17 09:16:50.573928: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/y_/6yh0ybp146v2ggxr5pqmm5mm0000gn/T/tmpvg42yt5q
2023-09-17 09:16:50.574935: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-09-17 09:16:50.574946: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/y_/6yh0ybp146v2ggxr5pqmm5mm0000gn/T/tmpvg42yt5q
2023-09-17 09:16:50.578315: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-09-17 09:16:50.605667: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [115]:
with tf.device('/cpu:0'):

    def batchPredict(csv_array):
        probs = []

        # define intepreter for quantized modell
        intepreter = tf.lite.Interpreter(model_path="my_model.tflite")
        intepreter.allocate_tensors()
        #get outputs and inputs details
        output = intepreter.get_output_details()
        input = intepreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']
        
     
        # output_index = output[0]['index']
        # input_index = input[0]['index']
        
        #set input ->invoke -> access outputs
        list_of_images_of_a_singlevid = np.array(csv_array)

        #run through model and predict the output
        for instance in list_of_images_of_a_singlevid:
            reshaped_instance = np.expand_dims(np.expand_dims(instance, axis=0), axis=0)
            intepreter.set_tensor(input_index, reshaped_instance)
            intepreter.invoke()
            output_data = intepreter.get_tensor(output_index)
            probs.append(output_data[0][0])
            
        return probs
    
    right_data_test = np.float32(right_data)
    drop_fast_data_test = np.float32(drop_fast_data)
    print(batchPredict(drop_fast_data_test))





[array([0.02368864, 0.14093748, 0.10072838, 0.73464555], dtype=float32), array([0.06345551, 0.1979844 , 0.25926346, 0.47929665], dtype=float32), array([0.03591906, 0.11458869, 0.05905516, 0.79043704], dtype=float32), array([0.2535136 , 0.37524828, 0.16230814, 0.20893   ], dtype=float32), array([0.10147759, 0.34575492, 0.12509008, 0.4276774 ], dtype=float32), array([0.06315286, 0.27585843, 0.09351391, 0.56747484], dtype=float32), array([0.02706174, 0.26471692, 0.3664648 , 0.34175652], dtype=float32), array([0.09552024, 0.49504775, 0.20312628, 0.20630573], dtype=float32), array([0.06700803, 0.23236483, 0.39410558, 0.30652162], dtype=float32), array([0.00097353, 0.10406409, 0.89103824, 0.00392418], dtype=float32), array([0.06438878, 0.22989573, 0.3622091 , 0.34350643], dtype=float32), array([0.06450433, 0.17961386, 0.07349735, 0.68238443], dtype=float32), array([0.05867273, 0.20876576, 0.10652524, 0.6260363 ], dtype=float32), array([0.04355145, 0.41054228, 0.18204805, 0.36385822], dtype=f